In [1]:
pip install --upgrade pandas numpy scikit-learn matplotlib seaborn statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 25.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 27.2 MB/s eta 0:00:0000:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
boston_df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv', usecols=lambda x: x != 'Unnamed: 0')

In [4]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,5.33,36.2


In [5]:
boston_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  LSTAT    506 non-null    float64
 12  MEDV     506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


In [6]:
boston_df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
LSTAT      0
MEDV       0
dtype: int64

In [7]:
boston_df.duplicated().sum()

np.int64(0)

In [8]:
boston_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


In [9]:
boston_df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT', 'MEDV'],
      dtype='object')

SELECCIÓN DE CARACTERÍSTICAS

In [10]:
x_var=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'LSTAT']
X = boston_df[x_var]  # Características
y = pd.DataFrame(boston_df['MEDV'])  # Variable objetivo (precio de las casas)

In [11]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFE


# Dividir el dataset reducido en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Aplicar RFE en el conjunto de entrenamiento
selector = RFE(model, n_features_to_select=4, step=1)
selector = selector.fit(X_train, y_train)

# Transformar los datos de entrenamiento y prueba
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Ajustar el modelo al conjunto de entrenamiento con las características seleccionadas
model.fit(X_train_selected, y_train)

# Predecir los valores en el conjunto de prueba
y_pred = model.predict(X_test_selected)

In [12]:
# Calcular las métricas
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
#mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R-squared: {r2:.2f}")
#print(f"Mean Absolute Percentage Error: {mape:.2f}%")

Mean Squared Error: 11.11
Root Mean Squared Error: 3.33
Mean Absolute Error: 2.26
R-squared: 0.85


In [13]:
# Mostrar las características seleccionadas
selected_features = selector.support_
feature_ranking = selector.ranking_

print("Características seleccionadas y Ranking:")
for i, col in enumerate(X.columns):
    print(col,'------->',selected_features[i],'-------------->',feature_ranking[i])

Características seleccionadas y Ranking:
CRIM -------> True --------------> 1
ZN -------> False --------------> 8
INDUS -------> False --------------> 6
CHAS -------> False --------------> 9
NOX -------> False --------------> 4
RM -------> True --------------> 1
AGE -------> False --------------> 5
DIS -------> True --------------> 1
RAD -------> False --------------> 7
TAX -------> False --------------> 3
PTRATIO -------> False --------------> 2
LSTAT -------> True --------------> 1


MODELO FINAL CON CARACTERISTICAS SELECCIONADAS

In [14]:
var_sel=['CRIM','RM','DIS','LSTAT']
X_sel=boston_df[var_sel]
X_sel.head()

,CRIM,RM,DIS,LSTAT
0,0.00632,6.575,4.0900,4.98
1,0.02731,6.421,4.9671,9.14
2,0.02729,7.185,4.9671,4.03
3,0.03237,6.998,6.0622,2.94
4,0.06905,7.147,6.0622,5.33


In [15]:
# Dividir el dataset reducido en conjunto de entrenamiento y prueba
X_trainf, X_testf, y_trainf, y_testf = train_test_split(X_sel, y, test_size=0.2, random_state=42)

# Crear el modelo
modelf = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenamiento Modelo con las características seleccionadas
modelf.fit(X_trainf, y_trainf)

# Predecir los valores en el conjunto de prueba
y_predf = model.predict(X_testf)

In [16]:
# Calcular las métricas
msef = mean_squared_error(y_testf, y_predf)
rmsef = np.sqrt(msef)
maef = mean_absolute_error(y_testf, y_predf)
r2f = r2_score(y_testf, y_predf)

print(f"Mean Squared Error: {msef:.2f}")
print(f"Root Mean Squared Error: {rmsef:.2f}")
print(f"Mean Absolute Error: {maef:.2f}")
print(f"R-squared: {r2f:.2f}")

Mean Squared Error: 11.11
Root Mean Squared Error: 3.33
Mean Absolute Error: 2.26
R-squared: 0.85


OPTIMIZACION DEL MODELO

In [17]:
# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [10, 50, 100, 200,300],
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_depth': [None, 5, 10, 15,20],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2'],
    }

# Crear un objeto RandomizedSearchCV
model_rs = RandomizedSearchCV(RandomForestRegressor(), param_distributions=param_grid, cv=10, scoring='neg_mean_squared_error', n_iter=40, verbose=1)
model_rs.fit(X_trainf, y_trainf)

Fitting 10 folds for each of 40 candidates, totalling 400 fits


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=40,
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'poisson'],
                                        'max_depth': [None, 5, 10, 15, 20],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 5, 10],
                                        'min_samples_split': [2, 5, 10, 20],
                                        'n_estimators': [10, 50, 100, 200,
                                                         300]},
                   scoring='neg_mean_squared_error', verbose=1)

In [18]:
print("Mejor puntuación:", model_rs.best_score_)
print("Mejores hiperparámetros:", model_rs.best_params_)

Mejor puntuación: -15.364832835655482
Mejores hiperparámetros: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None, 'criterion': 'absolute_error'}


In [19]:
# Crear el modelo
modelo_f = RandomForestRegressor(n_estimators=50, min_samples_split= 2, min_samples_leaf=1, max_features='log2', max_depth= 15, 
                               criterion='poisson',random_state=42)

# Entrenamiento Modelo con las características seleccionadas
modelo_f.fit(X_trainf, y_trainf)

# Predecir los valores en el conjunto de prueba
y_predf_rs = model.predict(X_testf)

GUARDADO DEL MODELO 

In [20]:
from pickle import dump

dump(modelo_f, open("random_forest_regressor_42.sav", "wb"))

# De esta forma cargo el archivo
#from pickle import load
#with open("random_forest_regressor_42.sav", "rb") as f:
#    loaded_model = load(f)